In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.utils import resample
from sklearn.model_selection import KFold

In [ ]:
# Load the county sentiment dataset
county_path = '../result-regression-data/county_sentiment.csv'
county = pd.read_csv(county_path)

county

In [ ]:
# Rename columns 'county' to 'County_Name' and 'state' to 'State_Name'
county.rename(columns={'county': 'County_Name', 'state': 'State_Name'}, inplace=True)

# Append 'County' to each value in the 'County_Name' column
county['County_Name'] = county['County_Name'] + ' County'

county

In [ ]:
# Load the 'socioeconomic' data
socioeconomic_path = '../socio-economic-data/county socioeconomic.csv'
socioeconomic = pd.read_csv(socioeconomic_path)

socioeconomic

In [ ]:
# Select only the useful columns from the socioeconomic df
socioeconomic_columns = [
    'County_Name', 'State_Name', 'Democrat_R', 'Republican_R', 'Total_Population', 'Median_income',
    'GINI', 'No_Insurance_R', 'Household_Below_Poverty_R', 'HISPANIC_LATINO_R', 'White_R', 'Black_R', 
    'Indian_R', 'Asian_R', 'Under_18_R', 'Bt_18_44_R', 'Bt_45_64_R', 'Over_65_R', 'Male_R', 
    'Bachelor_R', 'Education_Degree_R', 'Population_Density', 'Unemployed_R'
]
socioeconomic = socioeconomic[socioeconomic_columns]

socioeconomic

In [ ]:
# Merge the 'socioeconomic' and 'county' DataFrames on 'County_Name' and 'State_Name'
df = pd.merge(socioeconomic, county, on=['County_Name', 'State_Name'], how='inner')
file_path = '../result-regression-data/gis_plot_data.csv'
df.to_csv(file_path, index=False)
df

In [ ]:
# Create the figure and axis with matching figure size
fig, ax = plt.subplots(figsize=(6, 4))
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.weight'] = 'bold'
# Create boxplots with custom colors and black lines
sns.boxplot(data=df_merged[['Pre-COVID_average_sentiment', 
                           'During-COVID_average_sentiment', 
                           'Post-COVID_average_sentiment']],
            ax=ax,
            palette=['lightblue', 'lightpink', '#D3D3D3'],
            boxprops=dict(edgecolor='black'),  # Box edge color
            whiskerprops=dict(color='black'),  # Whisker color
            capprops=dict(color='black'),      # Cap color
            medianprops=dict(color='black'),   # Median line color
            flierprops=dict(color='black', markeredgecolor='black'))  # Outlier color
# Set labels and titles with bold formatting
ax.set_ylabel('Sentiment Score(-1: Shortage, 1: No Shortage)', fontsize=12, fontweight='bold')
# Set x-tick labels with bold formatting
ax.set_xticklabels(['Pre-Pandemic', 'Peak-Pandemic', 'Post-Peak'], 
                   fontsize=12, 
                   fontweight='bold')  # Added bold to tick labels
# Adjust tick parameters and make tick labels bold
for tick in ax.get_xticklabels() + ax.get_yticklabels():
    tick.set_fontweight('bold')
# Add grid lines for better readability
ax.grid(True, linestyle='--', alpha=0.5)
# Adjust layout
plt.tight_layout()
# Save the figure with increased dpi to improve clarity
plt.savefig('sentiment_boxplot.pdf', format='pdf', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Extract independent variables and perform standardization
X = df[['Democrat_R', 'Republican_R', 'Total_Population', 'Median_income', 'GINI', 'No_Insurance_R', 
        'Household_Below_Poverty_R', 'HISPANIC_LATINO_R', 'White_R', 'Black_R', 'Indian_R', 'Asian_R', 
        'Under_18_R', 'Bt_18_44_R', 'Bt_45_64_R', 'Over_65_R', 'Male_R', 'Bachelor_R',
        'Education_Degree_R', 'Population_Density', 'Unemployed_R']].fillna(0)

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

y_pre = df['Pre-COVID_average_sentiment'].fillna(0)
y_during = df['During-COVID_average_sentiment'].fillna(0)
y_post = df['Post-COVID_average_sentiment'].fillna(0)

# Create DataFrame after standardization
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Function to calculate VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Calculate and display VIF
vif_data = calculate_vif(X_scaled_df)

vif_data

In [ ]:
# PLS

# Define root_mean_squared_error function
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# PLS regression function with p-value and standard error
def pls_regression(X, y, n_permutations=1000):
    pls = PLSRegression(n_components=5)
    pls.fit(X, y)
    coefficients = pls.coef_.ravel()
    
    # R² and RMSE calculation
    y_pred = pls.predict(X)
    r2 = r2_score(y, y_pred)
    rmse = root_mean_squared_error(y, y_pred)
    
    # Permutation testing for p-values and standard errors
    p_values, std_errors = [], []
    for i in range(X.shape[1]):
        original_score = np.corrcoef(X[:, i], y.ravel())[0, 1]
        permuted_scores = [np.corrcoef(X[:, i], resample(y).ravel())[0, 1] for _ in range(n_permutations)]
        p_value = np.sum(np.abs(permuted_scores) >= np.abs(original_score)) / n_permutations
        std_error = np.std(permuted_scores)
        p_values.append(p_value)
        std_errors.append(std_error)
    
    return coefficients, p_values, std_errors, r2, rmse


y_pre = df['Pre-COVID_average_sentiment'].fillna(0)
y_during = df['During-COVID_average_sentiment'].fillna(0)
y_post = df['Post-COVID_average_sentiment'].fillna(0)

# Run PLS regression for each period
results = {}
for period, y in zip(['Pre', 'During', 'Post'], [y_pre, y_during, y_post]):
    coeffs, pvals, stderrs, r2, rmse = pls_regression(X_scaled, y.values.reshape(-1, 1))
    results[period] = (coeffs, pvals, stderrs)
    print(f'{period}-Pandemic R²: {r2:.3f}')
    print(f'{period}-Pandemic RMSE: {rmse:.3f}')

# Create DataFrames for each period
columns = X.columns
df_pre = pd.DataFrame({'Coefficient': results['Pre'][0], 
                       'P-Value': results['Pre'][1], 
                       'Standard Error': results['Pre'][2]}, index=columns).add_prefix('Pre-Pandemic ')

df_during = pd.DataFrame({'Coefficient': results['During'][0], 
                          'P-Value': results['During'][1], 
                          'Standard Error': results['During'][2]}, index=columns).add_prefix('During-Pandemic ')

df_post = pd.DataFrame({'Coefficient': results['Post'][0], 
                        'P-Value': results['Post'][1], 
                        'Standard Error': results['Post'][2]}, index=columns).add_prefix('Post-Pandemic ')

# Concatenate results and round to 3 decimal places
df_pls = pd.concat([df_pre, df_during, df_post], axis=1).round(3)

df_pls

In [ ]:
# During - Before

# Define y_diff for the difference between during-pandemic and pre-pandemic sentiment
y_diff_during_before = df['During-COVID_average_sentiment'].fillna(0) - df['Pre-COVID_average_sentiment'].fillna(0)

# Function to perform PLS regression without cross-validation, only with permutation testing
def pls_regression_with_permutation(X, y, n_permutations=1000):
    pls = PLSRegression(n_components=5)
    pls.fit(X, y)
    coefficients = pls.coef_.ravel()

    # R² and RMSE calculation
    y_pred = pls.predict(X)
    r2 = r2_score(y, y_pred)
    rmse = root_mean_squared_error(y, y_pred)
    
    # Permutation testing for p-values and standard errors
    p_values = []
    std_errors = []
    for i in range(X.shape[1]):
        original_score = np.corrcoef(X[:, i], y.ravel())[0, 1]
        permuted_scores = [np.corrcoef(X[:, i], resample(y).ravel())[0, 1] for _ in range(n_permutations)]
        p_value = (np.sum(np.abs(permuted_scores) >= np.abs(original_score))) / n_permutations
        std_error = np.std(permuted_scores)
        p_values.append(p_value)
        std_errors.append(std_error)
    
    return coefficients, p_values, std_errors, r2, rmse

# Run PLS regression for the difference
coeff_diff, pval_diff, stderr_diff, r2_diff, rmse_diff = pls_regression_with_permutation(X_scaled, y_diff_during_before.values.reshape(-1, 1))

# Print R² and RMSE results
print(f'During-Before R²: {r2_diff:.3f}')
print(f'During-Before RMSE: {rmse_diff:.3f}')

# Create a DataFrame to hold the results
columns = X.columns
df_diff_during_before = pd.DataFrame({'Coefficient': coeff_diff, 'P-Value': pval_diff, 'Standard Error': stderr_diff}, index=columns).round(3)

df_diff_during_before

In [ ]:
# Post - During
# Define y_diff for the difference between Post-Pandemic and During-Pandemic sentiment
y_diff_post_during = df['Post-COVID_average_sentiment'].fillna(0) - df['During-COVID_average_sentiment'].fillna(0)

# Function to perform PLS regression without cross-validation, with permutation testing
def pls_regression_with_permutation_only(X, y, n_permutations=1000):
    pls = PLSRegression(n_components=5)
    pls.fit(X, y)
    coefficients = pls.coef_.ravel()

    # R² and RMSE calculation
    y_pred = pls.predict(X)
    r2 = r2_score(y, y_pred)
    rmse = root_mean_squared_error(y, y_pred)
    
    # Permutation testing for p-values and standard errors
    p_values = []
    std_errors = []
    for i in range(X.shape[1]):
        original_score = np.corrcoef(X[:, i], y.ravel())[0, 1]
        permuted_scores = [np.corrcoef(X[:, i], resample(y).ravel())[0, 1] for _ in range(n_permutations)]
        p_value = (np.sum(np.abs(permuted_scores) >= np.abs(original_score))) / n_permutations
        std_error = np.std(permuted_scores)
        p_values.append(p_value)
        std_errors.append(std_error)
    
    return coefficients, p_values, std_errors, r2, rmse

# Run PLS regression for the difference
coeff_diff_post_during, pval_diff_post_during, stderr_diff_post_during, r2_diff_post_during, rmse_diff_post_during = pls_regression_with_permutation_only(X_scaled, y_diff_post_during.values.reshape(-1, 1))

# Print R² and RMSE results
print(f'Post-During R²: {r2_diff_post_during:.3f}')
print(f'Post-During RMSE: {rmse_diff_post_during:.3f}')

# Create a DataFrame to hold the results
columns = X.columns
df_diff_post_during = pd.DataFrame({'Coefficient': coeff_diff_post_during, 'P-Value': pval_diff_post_during, 'Standard Error': stderr_diff_post_during}, index=columns).round(3)

df_diff_post_during